In [1]:
#!pip install flair

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("flair/ner-english-large")

C:\Users\JOBIAK\anaconda3\lib\site-packages\huggingface_hub\file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-08-24 12:26:20,291 loading file C:\Users\JOBIAK\.flair\models\ner-english-large\07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-08-24 12:27:05,169 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [3]:
from bs4 import BeautifulSoup
import re
def convert(html):
    description= []
    empty = [""]
    empty[-1]
    empty_list=[]
    soup = BeautifulSoup(html, "lxml")
    tags_list = list(set([tag.name for tag in soup.find_all()]))
    tags_list = ["/"+i for i in tags_list]+tags_list
    tags_list.append("br/")
    tags_list.append("p/")
    for tag in soup.find_all():
        if (str(tag) in str(empty[-1])):
            continue
        else:
            empty.append(tag)
    for i in empty[1:]:
        empty_list.append(re.findall(r'<>[^>]*>.*?</[^>]*>(?:<[^>]*/>)?|[^<>]+', str(i)))
    for i in range(len(empty_list)):
            for j in range(len(empty_list[i])):
                if  not(empty_list[i][j].isspace()):
                    if empty_list[i][j] in tags_list:
                        continue
                    else:
                        description.append(empty_list[i][j]) 
    return description

In [4]:
html = """<p>Gopuff, a leading consumer technology platform delivering everyday essentials in minutes from local facilities, is looking for delivery partners to deliver through goDrive, an app that connects Delivery Driver Partners with customers. With its own centrally located facilities in every local market it serves, Gopuff delivers thousands of everyday needs, from cleaning supplies, home needs and OTC medications to food, drinks and more. It&apos;s quick and easy to EARN BIG!</p>\n <p><br/></p>\n <p>Why Partner with goDrive?</p>\n <p><br/></p>\n <p>Earn a per order commission + 100% of your tips</p>\n <p>Make a guaranteed hourly minimum* (varies by market)</p>\n <p>Choose your own schedule and deliver when you want</p>\n <p>No restaurants, no riders - deliver from a Gopuff facility, that&#x2019;s it!</p>\n <p>What you&#x2019;ll need to get started:</p>\n <p><br/></p>\n <p>Gotta be 21 years old!</p>\n <p>Valid U.S. driver&#x2019;s license</p>\n <p>Vehicle insurance &amp; registration</p>\n <p>Smartphone</p>\n <p>Willing &amp; able to pass a pre-paid alcohol delivery training course (where applicable)</p>\n <p>How it works:</p>\n <p><br/></p>\n <p>Login to the goDrive app from your local facility</p>\n <p>Grab a few orders</p>\n <p>Deliver to happy customers</p>\n <p>No previous delivery experience required!&#xA0;</p>\n <p><br/></p>\n <p>--</p>\n <p><br/></p>\n <p>goDrive is an app that connects Delivery Driver Partners with consumers who want deliveries from on-demand businesses like Gopuff. Partnering with goDrive is a great opportunity for anyone looking for a flexible schedule. Delivery Driver Partners will be independent contractors. Delivery Driver Partners must have a valid driver&apos;s license, and minimum auto insurance and complete a background check. Delivery Driver Partners will use their own reliable car, iPhone or Android phone and their data and text plan. Previous delivery experience is not required, but we encourage drivers from other delivery or ridesharing services to submit an application. Let&apos;s get you on the road. Apply today! *Hourly minimum if requirements met.</p>"""

In [5]:
org_data = convert(html)

In [6]:
html1 = """<html><body><p>Gopuff, a leading consumer technology platform delivering everyday essentials in minutes from local facilities, is looking for delivery partners to deliver through goDrive, an app that connects Delivery Driver Partners with customers. With its own centrally located facilities in every local market it serves, Gopuff delivers thousands of everyday needs, from cleaning supplies, home needs and OTC medications to food, drinks and more. It's quick and easy to EARN BIG!</p> <p><br/></p> <p>Why Partner with goDrive?</p> <p><br/></p> <p>Earn a per order commission + 100% of your tips</p> <p>Make a guaranteed hourly minimum* (varies by market)</p> <p>When you want, choose your own schedule</p> <p>No restaurants, no riders - deliver from a Gopuff facility, that’s it!</p> <p>What you’ll need to get started: </p> <p><br/></p> <p>Gotta be 21 years old!</p> <p>Valid U.S. driver’s license</p> <p>Vehicle insurance &amp; registration</p> <p>Smartphone</p> <p>Willing &amp; able to pass a pre-paid alcohol delivery training course (where applicable)</p> <p>How it works: </p> <p><br/></p> <p>Go to your local facility to sign into the app</p> <p>Grab a few orders</p> <p>Deliver to happy customers</p> <p>No previous delivery experience required! </p> <p><br/></p> <p>--</p> <p><br/></p> <p>goDrive is an app that connects Delivery Driver Partners with consumers who want deliveries from on-demand businesses like Gopuff. There is a great opportunity for anyone who needs a flexible schedule to partner with goDrive. Delivery Driver Partners will be independent contractors. Delivery Driver Partners need to be valid drivers' licenses, have minimum auto insurance and complete a background check. Delivery Driver Partners will use their own reliable car, iPhone or Android phone and their data and text plan. We encourage drivers from other delivery services to submit an application since previous delivery experience isn't required. Let's get you on the road. Apply today! *Hourly minimum if requirements met.</p></body></html>"""

In [7]:
mod_data = convert(html1)

In [8]:
org_desc = [j for i in org_data for j in i.split(". ")]
mod_desc = [j for i in mod_data for j in i.split(". ")]

In [9]:
from sentence_transformers import SentenceTransformer, util
def paracheck(orginal_input,modified_input):
    orginal_input.strip()
    modified_input.strip()
    if orginal_input.__eq__(modified_input):
        return 1
    else:
        similarity_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
        embedding_1=similarity_model.encode(orginal_input,convert_to_tensor=False)
        embedding_2=similarity_model.encode(modified_input, convert_to_tensor=False)
        similarity=util.pytorch_cos_sim(embedding_1, embedding_2).numpy()[0][0]
        return similarity

In [10]:
def sentence_length(org_senten,mod_senten):
    org_word = len(org_senten.split())
    mod_word = len(mod_senten.split())
    if org_word == mod_word:
        return [False,org_word,mod_word]
    else:
        return [True,org_word,mod_word]

In [11]:
def pres_asterisk(org_desc,mod_desc):
    if "*" in org_desc:
        if org_desc.__eq__(mod_desc):
            return False
        elif "*" in org_desc and "*" in mod_desc:
            if org_desc.count("*") == mod_desc.count("*"):
                at_first=0
                at_last=0
                at_middle=0
                c=org_desc.count("*")
                c1=mod_desc.count("*")
                i=0
                j=-1
                at_first1=0
                at_last1=0
                at_middle1=0
                while True:
                    if org_desc[i]=="*":
                        at_first+=1
                        i+=1
                    if org_desc[j]=="*" :
                        at_last+=1
                        j-=1
                    if org_desc[i]!="*" and org_desc[j]!="*":
                        break
                at_middle=c-at_first-at_last
                i=0
                j=-1
                while True:
                    if mod_desc[i]=="*":
                        at_first1+=1
                        i+=1
                    if mod_desc[j]=="*" :
                        at_last1+=1
                        j-=1
                    if mod_desc[i]!="*" and mod_desc[j]!="*":
                        break
                at_middle1=c1-at_first1-at_last1
                if at_middle>0:
                    indexes = [x for x, v in enumerate(org_desc) if v == '*']
                    index_weight=sum(indexes)
                    if (index_weight)//len(indexes) <len(org_desc)//2:
                        if at_first1==(at_middle+at_first):
                            return False
                        else:
                            return True
                    else:
                        if at_last1==(at_middle+at_last):
                            return False
                        else:
                            return True
                elif at_first!=at_first1:
                    return True
                elif at_last>=1 and (at_last!=at_last1):
                    return True
                else:
                    return False
            else:
                return True
    elif "*" in org_desc and "*" not in mod_desc or "*" not in org_desc and "*"  in mod_desc:
        return True
    else:
        return False

In [12]:
def presence_colon(org_sen,mod_sen):
    if ":" in org_sen and ":" in mod_sen:
        org_ind = org_sen.index(':')
        mod_ind = mod_sen.index(':')
        original_col_data = org_sen[:org_ind]
        modified_col_data = mod_sen[:mod_ind]
        org_count = org_sen.count(':')
        mod_count = mod_sen.count(':')
        if original_col_data == modified_col_data and org_count == mod_count:
            return False
        else:
            return True
    elif (":" in org_sen and ":" not in mod_sen) or (":" not in org_sen and ":" in mod_sen):
        return True
    else:
        return False

In [13]:
def Cname_Loc_compare(Original_desc,Modified_desc):
    ner_model = Sentence(Original_desc)
    tagger.predict(ner_model)
    l=ner_model.get_spans('ner')
    company_list_original=[i.text for i in l if i.score>0.99 and i.tag=="ORG"]
    Locations_original=[i.text for i in l if i.score>0.99 and i.tag=="LOC"]
    ner_model = Sentence(Modified_desc)
    tagger.predict(ner_model)
    l=ner_model.get_spans('ner')
    company_list_modified=[i.text for i in l if i.score>0.99 and i.tag=="ORG"]
    Locations_modified=[i.text for i in l if i.score>0.99 and i.tag=="LOC"]
    result=[]
    if len(company_list_original)!=len(company_list_modified):
        result.append(True)
    else:
        f=0
        for i in range(len(company_list_modified)):
            if company_list_original[i]!=company_list_modified[i]:
                f=1
                break
        if f==0:
            result.append(False)
        else:
            result.append(True)
    if len(Locations_original)!=len(Locations_modified):
        result.append(True)
    else:
        f=0
        for i in range(len(Locations_original)):
            if Locations_original[i]!=Locations_modified[i]:
                f=1
                break
        if f==0:
            result.append(False)
        else:
            result.append(True)
    return result

In [14]:
def fullstop(orginial_desc,modified_desc):
    original_count=orginial_desc.count('.')
    modified_count=modified_desc.count('.')
    difference=original_count-modified_count
    if original_count==modified_count:
        return [False,difference]
    else:
        return [True,difference]

In [15]:
para_list = [round(paracheck(i,j), 3) for i,j in zip(org_desc,mod_desc)]
paraphased_list = [i for i in para_list if i!=1]
paraphased_count = len(paraphased_list)
print(paraphased_count)
average = sum(paraphased_list)*100/len(paraphased_list)
print(average)

5
95.27999997138977


In [16]:
length_match = [(sentence_length(i,j)) for i,j in zip(org_desc,mod_desc)]
print(length_match)

[[False, 32, 32], [False, 31, 31], [False, 7, 7], [False, 4, 4], [False, 10, 10], [False, 8, 8], [True, 9, 7], [False, 12, 12], [False, 6, 6], [False, 5, 5], [False, 2, 2], [False, 2, 2], [False, 4, 4], [False, 1, 1], [False, 13, 13], [False, 3, 3], [True, 9, 10], [False, 4, 4], [False, 4, 4], [False, 5, 5], [False, 1, 1], [False, 19, 19], [True, 14, 16], [False, 7, 7], [False, 18, 18], [False, 19, 19], [True, 20, 17], [False, 6, 6], [False, 7, 7]]


In [17]:
asterisk_list = [(pres_asterisk(i,j)) for i,j in zip(org_desc,mod_desc)]
print(asterisk_list)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [18]:
cname_loc_list = [(Cname_Loc_compare(i,j)) for i,j in zip(org_desc,mod_desc)]
print(cname_loc_list)

[[False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False]]


In [19]:
colon_list = [(presence_colon(i,j)) for i,j in zip(org_desc,mod_desc)]
print(colon_list)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [20]:
fullstop_list = [(fullstop(i,j)) for i,j in zip(org_desc,mod_desc)]
print(fullstop_list)

[[False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0], [False, 0]]
